# HW2: Query Expansion

---




## Name: Mohammad Mahdi Heydari Nasab
## Student ID: 99105389

# Query expansion

In this section, we want to exapnd our query and start searching (finding results) for new query with has been expanded. We use tf-idf embedding implemented in homework 1 section 6 for namnak dataset. For evaluation you should test 10 arbitrary query and find 10 most relevant documents to query and sort them by your own idea and evaluate it with given methods in evaluation part.

1. In the cell below fill out methods below for implementing query expansion based on rocchio algorithm.

In [1]:
import math
import hazm
import json
from hazm import Normalizer
import numpy as np
import pandas as pd


class PersianDoc:
    def __init__(self, title, content, category, link, docID):
        self.title = title
        self.content = content
        self.category = category
        self.raw_content = content
        self.link = link
        self.docID = docID


def append_persian_doc_file(docs, i, docID, with_abstract):
    filename = 'namnak/namnak-' + str(i) + '.json'
    json_file = open(filename, encoding="utf8")
    data = json.load(json_file)
    for j in range(len(data)):
        doc_title = data[j]['title']
        doc_link = data[j]['link']
        if with_abstract:
            doc_content = data[j]['abstract']
        else:
            doc_content = ''
        for new_content in data[j]['paragraphs']:
            doc_content += new_content
            doc_content += ' '
        doc_category = data[j]['category']
        docs.append(PersianDoc(doc_title, doc_content, doc_category, doc_link, docID + j))
    return docID + j


persian_docs = list()
docID = 1
for i in range(1, 31):
    docID = append_persian_doc_file(persian_docs, i, docID, True)


def normalize_and_tokenize_persian_docs(docs, punctuations_string):
    normalizer = Normalizer()
    for doc in docs:
        doc.content = normalizer.normalize(doc.content)
        doc.content = hazm.word_tokenize(doc.content)
        doc.content = [word for word in doc.content if word not in punctuations_string]


punctuations = "``''?:!.,;،؛؟"
normalize_and_tokenize_persian_docs(persian_docs, punctuations)
persian_stopwords = set(hazm.stopwords_list())
for doc in persian_docs:
    doc.content = [word for word in doc.content if word not in persian_stopwords]
persian_lemmatizer = hazm.Lemmatizer()
for doc in persian_docs:
    doc.content = [persian_lemmatizer.lemmatize(word) for word in doc.content]

persian_df = pd.DataFrame()
contents = list()
for document1 in persian_docs:
    contents.append(document1.content)
persian_df['Content'] = contents
persian_df = pd.DataFrame({'Content': np.concatenate(persian_df.Content.values)})
all_persian_unique_words_in_docs = list(persian_df['Content'].value_counts().keys())


def make_positional_index(docs):
    positional_index = dict()
    for doc in docs:
        wordIndex = 1
        for word in doc.content:
            if word in positional_index:
                token_data = positional_index[word]
                if doc.docID in token_data[1]:
                    token_data[1][doc.docID].append(wordIndex)
                else:
                    token_data[0] += 1
                    token_data[1][doc.docID] = [wordIndex]
            else:
                positional_index[word] = [1, {doc.docID: [wordIndex]}]
            wordIndex += 1
    return positional_index


persian_positional_index = make_positional_index(persian_docs)
persian_tf_idf_matrix = np.zeros((len(all_persian_unique_words_in_docs), len(persian_docs)))
for i in range(len(all_persian_unique_words_in_docs)):
    positional_index = persian_positional_index[all_persian_unique_words_in_docs[i]]
    document_frequency = positional_index[0]
    for j in range(len(persian_docs)):
        if persian_docs[j].docID in positional_index[1]:
            term_frequency = len(positional_index[1][persian_docs[j].docID])
        else:
            term_frequency = 0
        persian_tf_idf_matrix[i][j] = term_frequency * math.log2(len(persian_docs) / document_frequency)
for j in range(len(persian_docs)):
    norm = np.linalg.norm(persian_tf_idf_matrix[:, j])
    if norm != 0:
        persian_tf_idf_matrix[:, j] /= norm


persian_word_and_wordID = dict()
wordID = 0
for key in all_persian_unique_words_in_docs:
    if key not in persian_word_and_wordID:
        persian_word_and_wordID[key] = wordID
        wordID += 1


def return_result(query, k):
    normalizer = Normalizer()
    query = normalizer.normalize(query)
    query = hazm.word_tokenize(query)
    query = [word for word in query if word not in punctuations]
    query = [word for word in query if word not in persian_stopwords]
    query = [persian_lemmatizer.lemmatize(word) for word in query]
    vector = np.zeros(len(all_persian_unique_words_in_docs))
    for word in query:
        if word in persian_word_and_wordID:
            idf = math.log2(len(persian_docs) / persian_positional_index[word][0])
            vector[persian_word_and_wordID[word]] += idf
    norm = np.linalg.norm(vector)
    if norm != 0:
        vector /= norm
    cosine_similarities = dict()
    for count in range(len(persian_docs)):
        cosine_similarities[count] = np.dot(persian_tf_idf_matrix[:, count], vector)
    cosine_similarities = sorted(cosine_similarities.items(), key=lambda x: x[1])
    relevant_titles = list()
    for count2 in range(k):
        relevant_doc = persian_docs[cosine_similarities[(len(cosine_similarities) - 1) - count2][0]]
        print(relevant_doc.title, "  ", relevant_doc.link)
        relevant_titles.append(relevant_doc.title)
    print("\n\n")
    return relevant_titles

def return_result_with_rocchio(query_vec, k):
    norm = np.linalg.norm(query_vec)
    if norm != 0:
        query_vec /= norm
    cosine_similarities = dict()
    for count in range(len(persian_docs)):
        cosine_similarities[count] = np.dot(persian_tf_idf_matrix[:, count], query_vec)
    cosine_similarities = sorted(cosine_similarities.items(), key=lambda x: x[1])
    relevant_titles = list()
    for count2 in range(k):
        relevant_doc = persian_docs[cosine_similarities[(len(cosine_similarities) - 1) - count2][0]]
        print(relevant_doc.title, "  ", relevant_doc.link)
        relevant_titles.append(relevant_doc.title)
    print("\n\n")
    return relevant_titles
    
def calculate_avg_relatives(qeury_vec, n=10):
    """Find vectors of ten most similar documents to our query vector with cosine similarity
       and calcualte average of these ten documents' vectors.

    Args:
        query_vec (array): vector representation for query

    Returns:
        avg_relatives (array): average of n most similiar documents' vector to our query vector
    """

    ### Begin your code
    cosine_similarities = dict()
    for count in range(len(persian_docs)):
        cosine_similarities[count] = np.dot(persian_tf_idf_matrix[:, count], qeury_vec)
    cosine_similarities = sorted(cosine_similarities.items(), key=lambda x: x[1])
    sum_of_vectors = np.zeros(len(qeury_vec))
    for count2 in range(n):
        sum_of_vectors += persian_tf_idf_matrix[:, cosine_similarities[(len(cosine_similarities) - 1) - count2][0]]
    avg_relatives = sum_of_vectors/n
    ### End your code
    return avg_relatives


def calculate_avg_non_relatives(qeury_vec, n=10):
    """Find vectors of ten least similar documents to our query vector with cosine similarity
       and calcualte average of these ten documents' vectors.

    Args:
        query_vec (array): vector representation for query

    Returns:
        avg_non_relatives (array): average of n least similiar documents' vectors to our query vector
    """

    ### Begin your code
    cosine_similarities = dict()
    for count in range(len(persian_docs)):
        cosine_similarities[count] = np.dot(persian_tf_idf_matrix[:, count], qeury_vec)
    cosine_similarities = sorted(cosine_similarities.items(), key=lambda x: x[1])
    sum_of_vectors = np.zeros(len(qeury_vec))
    for count2 in range(n):
        sum_of_vectors += persian_tf_idf_matrix[:, cosine_similarities[count2][0]]
    avg_non_relatives = sum_of_vectors / n
    ### End your code
    return avg_non_relatives


def roccio_query_expansion(query_vec, avg_r_vec, avg_nr_vec):
    """Calculate exapnded query with rocchio algorithm in lecture 10 section 3.

    Args:
        query_vec (array): vector representation for query
        avg_r_vec (array): vector representating average of related docuemnts caluclated by
                           'calculate_avg_relatives' method
        avg_nr_vec (array): vector representating average of related documents caluclated by
                            'calculate_avg_non_relatives' method

    Returns:
        expanded_query (array): new embedding for given query calculated by rocchio algorithm
    """

    ### Begin your code
    alpha = 1
    beta = 0.75
    gamma = 0.15
    expanded_query = alpha * query_vec + beta * avg_r_vec - gamma * avg_nr_vec
    ### End your code

    return expanded_query

In [2]:
queries = ['سلامت دندان', 
           'سفیدی دندان', 
           'عوارض چاقی', 
           'برنامه غذایی مناسب', 
           'علائم کرونا', 
           'برنامه مراقبت از پوست', 
           'علل سردرد مزمن', 
           'ترک اعتیاد', 
           'افزایش میل جنسی', 
           'درمان ریزش مو']
relevant_docs_for_queries = dict()
relevant_docs_for_queries['سلامت دندان'] = ['نسخه گیاهی بوعلی سینا برای داشتن دندان سالم ، زیبا و سفید',
                                            'با این مراقبت ها نیازی به مراجعه به دندانپزشک نیست',
                                            'اگر این نشانه ها را دیدید خیلی سریع به پزشک مراجعه کنید!',
                                            'عوارض و عواقب پر کردن دندان',
                                            'وقتی نخ دندان در سلامت دندان تاثیری ندارد !',
                                            'علامت اصلی پخش شدن عفونت دندان در بدن',
                                            'وقتی کشیدن دندان بهترین چاره است!!',
                                            'هیولای وحشتناک روی دندان پوسیده را از نزدیک ببینید + تصاویر',
                                            'روش های سریع برای درمان گیاهی و خانگی “حساسیت دندان”',
                                            'آسیب های جبران ناپذیر پودرهای سفید کننده دندان']
relevant_docs_for_queries['سفیدی دندان'] = ['راه سفید کردن دندانها',
                                            'آسیب های جبران ناپذیر پودرهای سفید کننده دندان',
                                            'روش ویژه ی سفیدی دندان در کمتر از پنج دقیقه "تضمینی"',
                                            'نسخه گیاهی بوعلی سینا برای داشتن دندان سالم ، زیبا و سفید',
                                            'داشتن دندان های سفید هالیوودی با سفید کننده های خانگی',
                                            'سفید کردن دندان با پوست چند میوه',
                                            'دلیل اصلی بدرنگی و زرد شدن دندان ها چیست؟',
                                            'سفید شدن دندان | سریعترین روش “سفید شدن دندان” درخانه با مواد طبیعی',
                                            '7 نکته کلیدی برای سفیدی دندان',
                                            'سفید کردن دندان با بیلیچینگ بهتره یا لامینت؟']
relevant_docs_for_queries['عوارض چاقی'] = ['تاثیر منفی چاقی در باروری و روابط زناشویی',
                                           'از عوارض و خطر پنهان چاقی خبر دارید؟',
                                           'آیا چاقی باعث ریزش موهایتان میشود؟',
                                           'بزرگترین دشمن سلامتی بدنت رو میشناسی؟',
                                           'مصرف مواد قندی باعث افسردگی میشود',
                                           'افراد چاق | افراد چاق باهوش ترند یا افراد لاغر',
                                           'فواید و عوارض عمل بای پس معده',
                                           'عوارض جبران ناپذیر جراحی های لاغری',
                                           'مصرف یک وعده غذایی پرحجم در یک روز کار درستی است؟',
                                           'بیماری و مشکلاتی که علت بزرگی شکم شماست']
relevant_docs_for_queries['برنامه غذایی مناسب'] = ['یک برنامه عالی برای سم زدایی بدن در یک روز!',
                                                   'مواد غذایی و تغذیه مناسب هر گروه خونی چیست؟+ باید و نبایدها',
                                                   'یک برنامه غذایی درست و درمون برای آقایان',
                                                   'پاکسازی کبد با رژیم غذایی سم زدای عالی',
                                                   'اینجوری غذا بخوری، هم پیر میشی هم زشت!',
                                                   'علائم فردی که رژیم غذایی ناسالم و بدی دارد',
                                                   'برنامه غذایی مفید برای کاهش اختلالات تیروئید',
                                                   'مواد غذایی با جربی مفید',
                                                   'بایدها و نبایدهای مهم تغذیه دانش آموزان برای موفقیت تحصیلی']
relevant_docs_for_queries['علائم کرونا'] = ['علائم و عوارض بلند مدت کرونا + حقایق ناگفته',
                                            'با این علائم شک نکنید کرونا دلتا دارید',
                                            'این علائم کرونا رو داشتی، فوراً به پزشک مراجعه کن',
                                            'علائمی که کرونا رو ترسناک تر میکنه',
                                            'علامت ابتلا به کرونا پس از تزریق واکسن',
                                            'تفاوت علائم کرونا با سرماخوردگی و آنفولانزا + لیست کامل',
                                            'علائم جدید کرونا ،کووید 19 رنگ عوض کرد',
                                            'کرونای طولانی مدت؛ علامتی که که حالا حالاها خوب نمیشه!',
                                            'عاملی که باعث خوب شدن کرونا طول بکشه',
                                            'علائم کرونایی که جدیدا شناسایی شده']
relevant_docs_for_queries['برنامه مراقبت از پوست'] = ['مراقبت از پوست در ایام عید با نکاتی که نمیدانید',
                                                      'آشنایی با روتین مراقبت از پوست در سن 30 سالگی',
                                                      'روتین 5 مرحله ای مراقبت از پوست بدن برای مبتدیان',
                                                      '5 اشتباه تکراری در مراقبت از پوست',
                                                      'علامت اصلی پوست ترکیبی + برنامه مراقبت و زیبایی',
                                                      'مراقبت پوست مردان و توصیه هایی برای حفظ زیبایی پوست مردان',
                                                      'روتین پوستی مخصوص خانم هایی که وقت ندارن',
                                                      'برای جذاب شدن به این برنامه مراقبت از پوست نیاز دارید',
                                                      'فرمول مراقبت از پوست برای اونهایی که سفر زیاد میرن']
relevant_docs_for_queries['علل سردرد مزمن'] = ['سردرد و علل و درمان های گیاهی و خانگی سردرد',
                                               'درباره علل انواع سردرد و درمان های فوری آنها',
                                               'چرا سرم درد میکنه؟ برای درمان میگرن و رهایی از سردرد چه باید کرد',
                                               'علت همه نوع “سر درد” + درمان های سریع خانگی هریک',
                                               'کدام سردردها را باید جدی گرفت',
                                               'از علت تا درمان سردرد های جنسی',
                                               'علل و علائم اصلی سردردهای تنشی مزمن+ پیشگیری',
                                               'علل اصلی سردرد ورزشی + پیشگیری و درمان',
                                               'علل اصلی سردرد صبحگاهی چیست ؟',
                                               'درد پشت سر چه دلایلی دارد و چگونه درمان می شود؟']
relevant_docs_for_queries['ترک اعتیاد'] = ['درد روانی و جسمانی ترک اعتیاد به انواع مواد + درمان گیاهی',
                                           'ترک اعتیاد مجلل بچه پولدارها + تصاویر',
                                           'همه و همه چیز درباره اعتیاد جنسی افراد متاهل + درمان',
                                           'شوهر شما معتاد جنسی است ؟! + راه ترک اعتیاد جنسی',
                                           'این حالات بدن از عوارض ترک سیگار است',
                                           '“ترک سیگار” با خوراکی های پیشنهادی متخصصان طب سنتی',
                                           'مراقبت های بعد از ترک سیگار برای کسب سلامتی',
                                           'ساده ترین و قطعی ترین راه برای ترک سیگار']
relevant_docs_for_queries['افزایش میل جنسی'] = ['کاهش میل جنسی ؛ دلیل اصلی بی میلی جنسی زنان و مردان',
                                                'تاثیر بادام و توت فرنگی در افزایش میل جنسی زن و مرد',
                                                'تقویت قوای جنسی مردان با نسخه جادویی طب سنتی',
                                                'مردم دنیا برای تقویت قوای جنسی چی میخورن؟',
                                                'تمام خوردنی های کاهش دهنده میل جنسی',
                                                'رژیم مخصوص افزایش میل جنسی زنان در یائسگی + روش های طلایی',
                                                'علت و ریشه اصلی بی میلی جنسی مردان + راه حلی برای زنان',
                                                'تمام خوردنی های افزایش دهنده میل جنسی، 30 درمان بی میلی زن و مرد',
                                                'تغذیه افزایش میل جنسی، کدام برای خانم کدام برای آقا؟',
                                                'افزایش میل و توان جنسی با چند راه ساده']
relevant_docs_for_queries['درمان ریزش مو'] = ['دلیل اصلی ریزش مو در حمام و راه حل آن',
                                              'ریزش مو مردان|علت “ریزش مو مردان” و درمان ریزش مو',
                                              'ریزش موی زنان و دختران جوان چگونه درمان می شود؟',
                                              'درمان ریزش مو با نسخه های گیاهی هندی ها',
                                              'علائم “ریزش موی ارثی” و بهترین درمان ها برای ریزش مو ارثی',
                                              'بخور نخورهای مهم برای شمایی که “ریزش مو” دارید + تغذیه',
                                              'مواد غذایی موثر در تقویت مو',
                                              'دلیل ریزش موی نوجوان پسر و دختر + درمان خانگی',
                                              'علت قطع نشدن ریزش موهات رو اینجا پیدا میکنی']
relevant_titles_for_each_query_without_rocchio = list()
print('Relevant docs without rocchio: \n')
print('سلامت دندان: \n')
relevant_titles_for_each_query_without_rocchio.append(return_result('سلامت دندان', 10 ))
print('\nسفیدی دندان:\n')
relevant_titles_for_each_query_without_rocchio.append(return_result('سفیدی دندان', 10 ))
print('\nعوارض چاقی:\n')
relevant_titles_for_each_query_without_rocchio.append(return_result('عوارض چاقی', 10))
print('\nبرنامه غذایی مناسب:\n')
relevant_titles_for_each_query_without_rocchio.append(return_result('برنامه غذایی مناسب', 10))
print('\nعلائم کرونا:\n')
relevant_titles_for_each_query_without_rocchio.append(return_result('علائم کرونا', 10))
print('\nبرنامه مراقبت از پوست:\n')
relevant_titles_for_each_query_without_rocchio.append(return_result('برنامه مراقبت از پوست', 10))
print('\nعلل سردرد مزمن:\n')
relevant_titles_for_each_query_without_rocchio.append(return_result('علل سردرد مزمن', 10))
print('\nترک اعتیاد:\n')
relevant_titles_for_each_query_without_rocchio.append(return_result('ترک اعتیاد', 10))
print('\nافزایش میل جنسی:\n')
relevant_titles_for_each_query_without_rocchio.append(return_result('افزایش میل جنسی', 10))
print('\nدرمان ریزش مو:\n')
relevant_titles_for_each_query_without_rocchio.append(return_result('درمان ریزش مو', 10))

Relevant docs without rocchio: 

سلامت دندان: 

وقتی کشیدن دندان بهترین چاره است!!    https://namnak.com/when-should-tooth-extraction.p80540
هیولای وحشتناک روی دندان پوسیده را از نزدیک ببینید + تصاویر    https://namnak.com/هیولای-وحشتناک-روی-دندان-پوسیده.p26187
علت تغییر رنگ دندان در افراد مختلف    https://namnak.com/سفید-کردن-دندان.p64869
تاثیر دندان بر زیبایی و پیری صورت    https://namnak.com/teeth-effect-on-face.p79573
با این مراقبت ها نیازی به مراجعه به دندانپزشک نیست    https://namnak.com/dental-care-tips.p84142
رازهای فوق العاده جذاب درباره دندان ها    https://namnak.com/facts-about-teeth.p79756
3 مشکل جدی و تغییر دندان با افزایش سن     https://namnak.com/teeth-change-age.p80014
روش های سریع برای درمان گیاهی و خانگی “حساسیت دندان”    https://namnak.com/دکترعلت-حساسیت-دندان.p19355
نسخه گیاهی بوعلی سینا برای داشتن دندان سالم ، زیبا و سفید    https://namnak.com/داشتن-دندان-سالم.p21174
دلیل اصلی بدرنگی و زرد شدن دندان ها چیست؟    https://namnak.com/زرد-شدن-دندان.p9824




سفیدی دندان

ریزش موی غیر قابل برگشت و قابل برگشت کدامند؟    https://namnak.com/ریزش-مو.p52951
دلیل اصلی ریزش مو در حمام و راه حل آن    https://namnak.com/hair-falling-in-shower.p83116
دلیل اصلی ریزش مو در حمام و راه حل آن    https://namnak.com/hair-falling-in-shower.p83116
ریزش مو مردان|علت “ریزش مو مردان” و درمان ریزش مو    https://namnak.com/ریزش-مو-در-مردان.p66496
ریزش موی زنان و دختران جوان چگونه درمان می شود؟    https://namnak.com/ریزش-مو-زنان.p40945
20 دلیل و علت اصلی که باعث ریزش موهای زنان و مردان می شود    https://namnak.com/ریزش-مو.p66232
درمان ریزش مو با نسخه های گیاهی هندی ها    https://namnak.com/درمان-ریزش-مو.p23076
دلیل ریزش مو در دوران شیردهی    https://namnak.com/ریزش-مو-در-دوران-شیردهی.p39221
علت قطع نشدن ریزش موهات رو اینجا پیدا میکنی    https://namnak.com/hair-keeps-falling-out.p85247
لیست دارو هایی که سبب ریزش مو می شوند    https://namnak.com/medication-causing-hair-loss.p76661





2. In the cell below, again find out results but with expanded query and print your new results

In [3]:
# Print out new results with expanded query
relevant_titles_for_each_query_with_rocchio = list()
for query in queries:
    print(query ,':\n\n')
    normalizer = Normalizer()
    query = normalizer.normalize(query)
    query = hazm.word_tokenize(query)
    query = [word for word in query if word not in punctuations]
    query = [word for word in query if word not in persian_stopwords]
    query = [persian_lemmatizer.lemmatize(word) for word in query]
    vector = np.zeros(len(all_persian_unique_words_in_docs))
    for word in query:
        if word in persian_word_and_wordID:
            idf = math.log2(len(persian_docs) / persian_positional_index[word][0])
            vector[persian_word_and_wordID[word]] += idf
    norm = np.linalg.norm(vector)
    if norm != 0:
        vector /= norm
    avg_relatives = calculate_avg_relatives(vector, 10)
    avg_non_relatives = calculate_avg_non_relatives(vector, 10)
    expanded_query = roccio_query_expansion(vector, avg_relatives, avg_non_relatives)
    relevant_titles_for_each_query_with_rocchio.append(return_result_with_rocchio(expanded_query, 10))
    print('\n\n')

    

سلامت دندان :


وقتی کشیدن دندان بهترین چاره است!!    https://namnak.com/when-should-tooth-extraction.p80540
هیولای وحشتناک روی دندان پوسیده را از نزدیک ببینید + تصاویر    https://namnak.com/هیولای-وحشتناک-روی-دندان-پوسیده.p26187
با این مراقبت ها نیازی به مراجعه به دندانپزشک نیست    https://namnak.com/dental-care-tips.p84142
رازهای فوق العاده جذاب درباره دندان ها    https://namnak.com/facts-about-teeth.p79756
علت تغییر رنگ دندان در افراد مختلف    https://namnak.com/سفید-کردن-دندان.p64869
تاثیر دندان بر زیبایی و پیری صورت    https://namnak.com/teeth-effect-on-face.p79573
نسخه گیاهی بوعلی سینا برای داشتن دندان سالم ، زیبا و سفید    https://namnak.com/داشتن-دندان-سالم.p21174
3 مشکل جدی و تغییر دندان با افزایش سن     https://namnak.com/teeth-change-age.p80014
روش های سریع برای درمان گیاهی و خانگی “حساسیت دندان”    https://namnak.com/دکترعلت-حساسیت-دندان.p19355
دلیل اصلی بدرنگی و زرد شدن دندان ها چیست؟    https://namnak.com/زرد-شدن-دندان.p9824






سفیدی دندان :


راه سفید کردن دندانها    

ریزش موی غیر قابل برگشت و قابل برگشت کدامند؟    https://namnak.com/ریزش-مو.p52951
دلیل اصلی ریزش مو در حمام و راه حل آن    https://namnak.com/hair-falling-in-shower.p83116
دلیل اصلی ریزش مو در حمام و راه حل آن    https://namnak.com/hair-falling-in-shower.p83116
ریزش موی زنان و دختران جوان چگونه درمان می شود؟    https://namnak.com/ریزش-مو-زنان.p40945
ریزش مو مردان|علت “ریزش مو مردان” و درمان ریزش مو    https://namnak.com/ریزش-مو-در-مردان.p66496
20 دلیل و علت اصلی که باعث ریزش موهای زنان و مردان می شود    https://namnak.com/ریزش-مو.p66232
درمان ریزش مو با نسخه های گیاهی هندی ها    https://namnak.com/درمان-ریزش-مو.p23076
دلیل ریزش مو در دوران شیردهی    https://namnak.com/ریزش-مو-در-دوران-شیردهی.p39221
علت قطع نشدن ریزش موهات رو اینجا پیدا میکنی    https://namnak.com/hair-keeps-falling-out.p85247
لیست دارو هایی که سبب ریزش مو می شوند    https://namnak.com/medication-causing-hair-loss.p76661








3. Evaluate new results with given methods in evaluation part and compare it with results for plain query.

In [4]:
from evaluation import *

def make_actual_list(titles):
    actual_list = list()
    for j in range(len(persian_docs)):
        if persian_docs[j].title in titles:
            actual_list.append(True)
        else:
            actual_list.append(False)
    return actual_list
def make_predicted_list(titles):
    predicted_list = list()
    for j in range(len(titles)):
        for k in range(len(persian_docs)):
            if titles[j] == persian_docs[k].title:
                predicted_list.append(k)
    return predicted_list
                

# Calculate diffrent metrices based on your problem for you exapnded queries

from evaluation import get_precision_k, get_map_k
for i in range(len(queries)):
    actual_list = make_actual_list(relevant_docs_for_queries[queries[i]])
    y_predicted_without_rocchio = make_predicted_list(relevant_titles_for_each_query_without_rocchio[i])
    y_predicted_with_rocchio = make_predicted_list(relevant_titles_for_each_query_with_rocchio[i])
    print('Query: ', queries[i], ':')
    print('Presicion at 10 without rocchio: ', get_precision_k(actual_list, y_predicted_without_rocchio, 10))
    print('Presicion at 10 with rocchio: ', get_precision_k(actual_list, y_predicted_with_rocchio, 10))
    print('MAP at 10 without rocchio: ', get_map_k(actual_list, y_predicted_without_rocchio, 10))
    print('MAP at 10 with rocchio: ', get_map_k(actual_list, y_predicted_with_rocchio, 10))
    print('\n\n')
    

Query:  سلامت دندان :
Presicion at 10 without rocchio:  0.5
Presicion at 10 with rocchio:  0.5
MAP at 10 without rocchio:  0.7311111111111112
MAP at 10 with rocchio:  0.8253968253968254



Query:  سفیدی دندان :
Presicion at 10 without rocchio:  0.7
Presicion at 10 with rocchio:  0.7
MAP at 10 without rocchio:  0.9478458049886621
MAP at 10 with rocchio:  0.8976190476190476



Query:  عوارض چاقی :
Presicion at 10 without rocchio:  0.6
Presicion at 10 with rocchio:  0.6
MAP at 10 without rocchio:  0.5425925925925925
MAP at 10 with rocchio:  0.594047619047619



Query:  برنامه غذایی مناسب :
Presicion at 10 without rocchio:  0.1
Presicion at 10 with rocchio:  0.2
MAP at 10 without rocchio:  0.16666666666666666
MAP at 10 with rocchio:  0.2



Query:  علائم کرونا :
Presicion at 10 without rocchio:  0.6
Presicion at 10 with rocchio:  0.6
MAP at 10 without rocchio:  0.7999999999999999
MAP at 10 with rocchio:  0.7999999999999999



Query:  برنامه مراقبت از پوست :
Presicion at 10 without rocchio: